In [2]:
#import modules
# import serial

import cv2
import numpy as np

import random
from pydub.generators import Sine
from pydub.generators import Sawtooth
from pydub.effects import low_pass_filter
from pydub.playback import play
import threading
import multiprocessing as mp
import time


In [3]:
#init
# ser = serial.Serial(port='COM3', baudrate=57600)

# set windows size
height = 480
width = 640
volume = 50

faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eyeCascade = cv2.CascadeClassifier('haarcascade_eye.xml')
# eyeCascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
lock = threading.Lock()
# set camera
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(3,width)
cam.set(4,height)

def CloseAll():
    ser.close()
#     cam.release()
#     cv2.destroyAllWindows()

In [4]:
# Arduino to key numbers list
def SerialReceiver():
    line = ser.readline().decode('utf-8').rstrip()
    dist = line.split()
    return dist

def Dist2Chord(dist):
    l=int(dist[0])
    r=int(dist[1])
    lb = 5  #lower bound
    ub = 50 #upper bound
    if(l<lb or l>ub):
        return 0
    elif(r<lb or r>ub):
        return 0
    else:
        det = ub-lb
        ln = 9 #status num
        rn = 9 #status num
        dl = det//ln
        dr = det//rn
        base = l//dl
        chord = r//dr
        ls = [base, chord]
        return ls

def KeyNDecoder(code):
    if code==0:
        return 0
    elif code[0]==0:
        return 0
    keys = []
    keys.clear()
    base = code[0]-1
    symb = code[1]-1
    
    #base table
    basenote=28
    bt = [0,2,4,5,7,9,11,12,14]
    #chord table
    ct = [
            [0,4,7,12],
            [0,4,7,11],
            [0,4,7,10],
            [0,3,7,12],
            [0,3,7,10],
            [0,3,6,10],
            [0,7,12],
            [0,5,7,12],
            [0,2,7,12]
         ]
    #https://en.wikipedia.org/wiki/Chord_(music)
    
    
    base = bt[base]+basenote
    for i in ct[symb]:
        note = base + i
        keys.append(note)
    return keys

def Arduino2Keys():
    dist = SerialReceiver()
    code = Dist2Chord(dist)
    keys = KeyNDecoder(code)
    return keys

# Note
# 1 2 3 4 5 6 7 8 9 = C3 D3 E3...A4 B4 C4 D4

# Chord

# Major
# 1 3 5 8  M
# 1 3 5 7  M7
# 5 7 2 4  7

# Minor
# 6 1 3 6  m
# 6 1 3 5  m7
# 7 2 4 6  phi

# Rock
# 1 5      power chord
# 1 4 5    sus4
# 1 2 5    sus2

def FixedChord(idx):
    #keys
    t0=[8,5,6,3,4,1,4,5]
    t1=[1,1,4,4,1,1,1,1]
    code = [t0[idx],t1[idx]]
    keys = KeyNDecoder(code)
    return keys

In [5]:
def play_chord_Sawtooth(notes):
    global volume
    frequencies = []
    sounds = []
    for note in notes:
        frequency = (2 ** (1 / 12 * (int(note) - 49))) * 440
        frequencies.append(frequency)
        sound = Sawtooth(freq = frequency).to_audio_segment(duration = 600)
        ##get volume
        lock.acquire()
        increase = volume / 2 - 25 
        lock.release()
        
        sound += increase
        mixed = sound
        mixed = low_pass_filter(mixed, random.uniform(800, 1000))
        sounds.append(mixed)
    chord = sounds[0]
    for sound in sounds[1:]:
        chord = chord.overlay(sound)
    play(chord)
#     return chord

In [5]:
def face_recognition():
    global volume
    volume = 50
    #stream
    while True:
        ret, img = cam.read()
        img = cv2.flip(img, 1)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=7,
            minSize=(100,100)
        )
        sum = 0
        eye_sum = 0
        num = 0
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h,x:x+w]
            roi_color = img[y:y+h,x:x+w]
            
            eyes = eyeCascade.detectMultiScale(
                roi_gray,
                scaleFactor= 1.5,
                minNeighbors=7,
                minSize=(5, 5),
            )
            for (ex, ey, ew, eh) in eyes:
                if(ey < h//3):
                    num = num + 1
                    sum = sum + ey + y
                    eye_sum = eye_sum + eh
                    cv2.circle(roi_color, (ex + ew//2, ey + eh//2), 5, (0, 0, 255), -1) 
        if(num != 0):
            eye_height = sum/num
            lock.acquire()
            volume = 50 - (eye_height - height/2+(2*eye_sum)/(3*num))*400/(height)
            lock.release()
        if(volume>100):
            volume = 100
        elif(volume<0):
            volume = 0
        cv2.putText(img, "volume: {:.1f}".format(volume), (10, 40), cv2.FONT_HERSHEY_SIMPLEX,  1, (0, 0, 255), 4, cv2.LINE_AA)
        cv2.ellipse(img, (width//2,height//2),(120,180),0,0,360,(0,255,0),2)
        cv2.imshow('test',img)
        k = cv2.waitKey(30) & 0xff
        if not (cv2.getWindowProperty('test',cv2.WND_PROP_VISIBLE)):
            break
        if k ==27:
            break

In [9]:
if __name__=='__main__':
    i=0
    #=========face recognition================
    t_face = threading.Thread(target=face_recognition, args=())
    t_face.start()
    while True:

        #=========Part 1 Get Key numbers==========        
        #A Arduino ver
        #keys = Arduino2Keys()
        #print(keys)
        
        #B Fixed chord generator
#         keys= FixedChord(i)
#         i+=1
#         if i==8:
#             i=0
        #=========================================
        
        #Set the volumn
        #volumn = GetVol()
        
        #Play the sound
        t_list = []
        for i in range(8):
            keys= FixedChord(i)
            if keys != 0:
                #Play(keys)
                t = threading.Thread(target=play_chord_Sawtooth, args=(keys, ))
                t_list.append(t)
                t.start()
                time.sleep(0.45)
                #play_chord_Sawtooth(keys)    
        
#         for t in t_list:
#             t.join()
        t_list.clear()


NameError: name 'FixedChord' is not defined

In [ ]:
#Run this cell before closing the tab or uploading the Arduino code

CloseAll()